In [18]:
video_url = "https://youtu.be/ZOJCatQ9fQU?si=smLS1mZeZnwkVTM0"

# 기능.1 : 김형섭

In [19]:
import os
import yt_dlp
from datetime import datetime
# from fastapi import HTTPException

OUTPUT_FOLDER = "/home/wanted-1/potenup-workspace/Project/project3/team2/02_WebFramework/output_folder"

In [20]:
def download_audio(video_url: str):
    """유튜브 비디오에서 오디오 추출하여 output_folder에 저장"""

    os.makedirs(OUTPUT_FOLDER, exist_ok=True)       # output_folder 생성

    # 타임스탬프 추가 (파일명 중복 방지)
    # timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    audio_filename = "extracted_audio"
    audio_path = os.path.join(OUTPUT_FOLDER, audio_filename)

    # 유튜브 오디오 다운로드 옵션 설정
    ydl_opts = {
        "format": "bestaudio/best",
        "outtmpl": audio_path,                      # 저장될 경로 지정
        "postprocessors": [{
            "key": "FFmpegExtractAudio",
            "preferredcodec": "mp3",
            "preferredquality": "192"
        }],
    }

    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([video_url])               # 유튜브에서 오디오 다운로드
    except Exception as e:
        print(e)
        # raise HTTPException(status_code=500, detail=f"오디오 추출 실패: {str(e)}")

    return audio_path                               # 다운로드된 파일 경로 반환

In [21]:
audio_path = download_audio(video_url)

[youtube] Extracting URL: https://youtu.be/ZOJCatQ9fQU?si=smLS1mZeZnwkVTM0
[youtube] ZOJCatQ9fQU: Downloading webpage


[youtube] ZOJCatQ9fQU: Downloading tv client config
[youtube] ZOJCatQ9fQU: Downloading player 5ae7d525
[youtube] ZOJCatQ9fQU: Downloading tv player API JSON
[youtube] ZOJCatQ9fQU: Downloading ios player API JSON
[youtube] ZOJCatQ9fQU: Downloading m3u8 information
[info] ZOJCatQ9fQU: Downloading 1 format(s): 251
[download] Destination: /home/wanted-1/potenup-workspace/Project/project3/team2/02_WebFramework/output_folder/extracted_audio
[download] 100% of    3.82MiB in 00:00:00 at 21.19MiB/s    
[ExtractAudio] Destination: /home/wanted-1/potenup-workspace/Project/project3/team2/02_WebFramework/output_folder/extracted_audio.mp3
Deleting original file /home/wanted-1/potenup-workspace/Project/project3/team2/02_WebFramework/output_folder/extracted_audio (pass -k to keep)


In [22]:
audio_path

'/home/wanted-1/potenup-workspace/Project/project3/team2/02_WebFramework/output_folder/extracted_audio'

# 기능.2 : 지서연

In [1]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

def load_asr_pipeline(model_id: str, torch_dtype,  device: str):
    model = AutoModelForSpeechSeq2Seq.from_pretrained(
        model_id,
        torch_dtype=torch_dtype,
        low_cpu_mem_usage=True,
        use_safetensors=True
    )
    model.to(device)
    processor = AutoProcessor.from_pretrained(model_id)
    
    asr_pipe = pipeline(
        "automatic-speech-recognition",
        model=model,
        tokenizer=processor.tokenizer,
        feature_extractor=processor.feature_extractor,
        torch_dtype=torch_dtype, #torch.float16,
        device=device,
    )
    return asr_pipe


/home/wanted-1/miniconda3/envs/whisper/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 

model_id = "openai/whisper-small"

load_asr_pipeline(model_id, torch_dtype, device)

Device set to use cuda:0


In [4]:
import re
from collections import defaultdict

def remove_extra_repeated_words(text: str) -> str:
    """
    동일한 한글 단어가 3번 이상 반복될 경우 2개만 남기고 나머지 삭제
    (숫자 및 특수문자는 유지)
    """
    word_count = defaultdict(int)
    word_positions = {}
    
    for match in re.finditer(r"[가-힣]+", text):
        word = match.group()
        word_count[word] += 1
        word_positions.setdefault(word, []).append(match.start())
    
    result = list(text)
    for word, positions in word_positions.items():
        if word_count[word] > 2:
            for i in range(2, len(positions)):
                start_idx = positions[i]
                for j in range(len(word)):
                    result[start_idx + j] = ""
    return "".join(result)


In [5]:
import torch
import gc
import os

def initialize_gpu():
    """
    GPU 캐시, IPC, 가비지 컬렉션 실행 후 GPU 동기화 및 초기화
    """
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()
    gc.collect()
    torch.cuda.synchronize()
    # os.system("nvidia-smi --gpu-reset")  # 필요시 주석 해제 (관리자 권한 필요)


In [6]:
#!/usr/bin/env python3
import os
# from gpu_utils import initialize_gpu
# from asr_model import load_asr_pipeline
# from text_utils import remove_extra_repeated_words

def main():
    # GPU 초기화
    initialize_gpu()
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    torch_dtype = torch.float16 

    model_id = "openai/whisper-small"
    asr_pipe = load_asr_pipeline(model_id, torch_dtype, device)

    # 음성 인식 설정 (forced_decoder_ids를 생략하여 내부 language 옵션 사용)
    generate_kwargs = {
        "max_new_tokens": 445,
        "num_beams": 1,
        "condition_on_prev_tokens": False,
        "compression_ratio_threshold": 1.35,
        "temperature": (0.0, 0.2, 0.4, 0.6, 0.8, 1.0),
        "logprob_threshold": -1.0,
        "no_speech_threshold": 0.6,
        "return_timestamps": True,
        # language 옵션은 내부적으로 처리하도록 합니다.
    }
    
    # 오디오 파일 경로
    audio_path = r'/home/wanted-1/potenup-workspace/Project/project3/team2/02_WebFramework/output_folder/extracted_audio.mp3'
    # language 옵션을 generate_kwargs에 포함하지 않고, 파이프라인 내부에서 설정하게 함.
    result = asr_pipe(audio_path, generate_kwargs={"language": "korean"}, return_timestamps=True)
    
    test_text = result.get("text", "")
    cleaned_text = remove_extra_repeated_words(test_text)
    print("처리된 텍스트:", cleaned_text)
    
    output_file = "processed_text.txt"
    with open(output_file, "w", encoding="utf-8") as f:
        f.write(cleaned_text)
    print(f"최종 텍스트가 '{output_file}'에 저장되었습니다.")

  


In [7]:
main()

Device set to use cuda:0
/home/wanted-1/miniconda3/envs/whisper/lib/python3.10/site-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
You have passed language=korean, but also have set `forced_decoder_ids` to [[1, None], [2, 50359]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of language=korean.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.


처리된 텍스트:  지난해 8월 어렵살이 미국 영주권을 취득했던 애틀렌타 한인 A 씨는 지난 6일 조지아주 연방 이민법원에서 열린 추방 재판에서 한국추방 판결을 받았습니다. 영주권 취득 6개월 만에 마주한 청천병력 같은 현실이었습니다. 이민 초기, 범죄에 휘말려 기소됐지만 단순 가담으로 정상이 참작돼 시령을 피했던 a 씨는 영주권 신청 과정에서도 이를 해명하기 위해 동분서주했습니다. 여러 차례의 서류 보완을 통해 해명 을 들은 연반 이민국은 미국 정착 에 문제가 없다며 그린 카드를 최종 발급해줬습니다. 꿈에 그리던 영주권을 받은 a 씨 는 두 달 후인 지난해 10월 한국에 있는 가족을 방문하기 위해 출국 했습니다. 그동안 신분 해결이 안되 비행기를 탈 수 없었기 때문에 10여 년 만에 밟은 모국당이었습니다. 하지만 2개월간의 한국체류를 마치고 12 중순 애틀렌타 국제공항을 통해 귀국하던 ac를 기다린 것은 생각지도 못했던 악몽이었습니다. 입국심사를 담당한 연방세관 국경보호국 cbp심사관은 ac의 영주권에 있는 외국인 등록번호를 확인하더니 2차 심사실로 끌고 갔습니다. 심사실에서 이전 범죄기록을 추금 받은 a   신청 과정에서 문제를 해결했고 이민국도 이를 받아들였다고 설명했지만 cbp 유원들은 a 씨에게 수갑을 채워 곧바로 조지아주 스튜어트 이민국 치소에 수감시켰습니다. a 씨에 대한 추방 재판은 이례적으로 신속하게 진행됐고 체포 두 달도 안 돼  판결이 내려졌습니다. a 씨를 면역했던  총영사관 관계자는 지난달 28일 한인 첫 불체자 케이스로 체포된 이면우 씨도 보름만 인 지난 14 추반 판결이 내려졌 다면서 이 구치소에 수감됐다가 이달 초 한국으로 송환된 다른 한 인도 영주권자였다고 말했습니다. 총영사관 관계자는 영주권자 등 합법 체류자라 할지라도 과거의 범죄 이력이 있으면 지금은 한국 등의 해외여행을 자제하는 것이 좋 다면서 합법체류 판정당국인 연방 이민국이 승인한  등의 합법 신분을 부여하더라도 단속당국인 cbp나  이민세관 단속국 icg 가  인정하지 

## 3. Gemma-2-2b-it

In [ ]:
import torch
import os
import re
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel, prepare_model_for_kbit_training, LoraConfig, get_peft_model

# GPU 0 으로 강제 설정
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 모델 불러오기 
BASE_MODEL = "google/gemma-2b-it"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=bnb_config,
    device_map={"": torch.cuda.current_device()}
)
base_model = prepare_model_for_kbit_training(base_model)

model = PeftModel.from_pretrained(base_model, "/home/wanted-1/potenup-workspace/Project/project3/team2/SY/Gemma/path/to/save/lora_adapters_ver1_law")
model.eval()  

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# 하이퍼파라미터 조정
generation_kwargs = {
    "max_length": 4096,           # 최대 생성 토큰 수 (요약 길이 제한)
    "min_length": 512,            # 최소 생성 토큰 수
    "do_sample": True,           # 샘플링 방식 사용
    "temperature": 0.5,          # 낮은 값은 결정론적, 높은 값은 다양성 증가
    "top_k": 50,                 # 상위 k개 단어 내에서 샘플링
    "top_p": 0.95,               # 누적 확률 p 내 단어에서 샘플
}

# 텍스트 파일 경로 수정
txt_file_path = "/home/wanted-1/potenup-workspace/Project/project3/team2/SY/Gemma/processed_text.txt"  # 파일 경로를 수정하세요.
with open(txt_file_path, "r", encoding="utf-8") as f:
    original_text = f.read()

# 프롬프트 
detailed_prompt = """
After reading the original text below, please perform the following steps in order:

1. Translate the original text into English.
2. Summarize the translated text by extracting only the most important core points. Present each key point as a numbered item (for example, "1.", "2.", "3.", etc.). Write each point concisely and clearly.
3. Finally, translate the numbered summary into Korean.

Important:
- The final output must be only the summary in Korean.
- Use simple and direct language.
- Do not include any extra commentary, notes, or repeated information.
- Each numbered point must be on a separate line and end with a period.
- Do not include any bullet symbols (such as "*" or "-").
- Do not write '**Note:**'
- Do not include any notes, disclaimers, or additional comments. Only output the summary.
- Do not write any 'Note:'
- Avoid duplication of word

Original Text:
{original_text}

Summary:
"""

# 프롬프트 구성
prompt = detailed_prompt.format(original_text=original_text)

#  요약 생성
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
with torch.no_grad():
    output_tokens = model.generate(**inputs, **generation_kwargs)

# 후처리 
def post_process(text):
    # 앞뒤 공백 제거
    text = text.strip()
    # 불필요한 공백, 줄바꿈, 중복 문장 부호 제거 (예시: 느낌표, 공백 등)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'!{2,}', '!', text)
    return text

generated_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
# 4. "Summary:" 이후의 텍스트만 추출하여 순수 요약문만 저장
if "Summary:" in generated_text:
    summary_only = generated_text.split("Summary:")[-1].strip()
else:
    summary_only = generated_text.strip()

# "Note:"가 있다면, 그 뒤의 모든 텍스트를 제거
if "Note:" in summary_only:
    summary_only = summary_only.split("Note:")[0].strip()


# .txt 파일로 저장 
with open("generated_summary.txt", "w", encoding="utf-8") as f:
    f.write(summary_only)

print("Summary Only:", summary_only)

with open("generated_summary.txt", "w", encoding="utf-8") as f:
    f.write(summary_only)
print("File 'generated_summary.txt' has been created.")